<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00


In [36]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy.optimize import root_scalar
from scipy.optimize import root
from scipy.special import expit, logit

In [3]:
p=Props(['Acetone','Ethanol', 'Isopropanol', 'Water'])

In [4]:
def gamma(x,T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G=np.exp(-p.NRTL_alpha*tau)
    xG = x@G
    xtauG_xG = (x@(tau*G))/xG
    return np.exp(xtauG_xG + x@((G*(tau - xtauG_xG[None,:])/xG[None,:]).T))

In [46]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [54]:
def bubbleP_NRTL(x, T):
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

def bubbleT_NRTL(x, P):

    def f(T):
        return bubbleP_NRTL(x,T)[0]-P

    #mole-fraction weighted boiling points of each component at P
    #boiling points determined via Clausius Clapeyron, using the Hvap at the normal bp
    #for each component.  p.Hvap returns the heat of vaporization of all components for each
    #temperature if an array of temperatures is given.
    Tguess=np.dot(x,1/(1/p.Tbn-np.log(P/101325)*R/np.diagonal(p.Hvap(p.Tbn))))

    T=root_scalar(f, x0=Tguess, method='secant').root
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)

    return T, Pi/P



In [55]:
z = np.array([0.1,0.2,0.3,0.4])
P = 3e5
R=8.314
bubbleT_NRTL(z,P)


(Array(377.80220917, dtype=float64),
 Array([0.25464361, 0.1853556 , 0.28067204, 0.27932875], dtype=float64))

In [43]:
def dewP_NRTL(y, T):
    def f(vec):
        P = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Pguess, xguess= dewP_ideal(z,T)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Pguess, logit(xguess)]
    res=root(f, v0)
    return res.x[0], expit(res.x[1:])

In [47]:
def dewT_NRTL(y, P):
    def f(vec):
        T = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Tguess, xguess= dewT_ideal(z,P)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Tguess, logit(xguess)]
    res=root(f, v0)
    return res.x[0], expit(res.x[1:])

In [48]:
dewP_NRTL(z, 373)

(215679.4631248687, array([0.02121585, 0.13299478, 0.17452966, 0.67125971]))

In [49]:
dewT_NRTL(z, 215679)

(372.99993756950886, array([0.02121583, 0.13299476, 0.17452962, 0.67125978]))

In [60]:
def flashPT(z, P, T):
    if P<dewP_NRTL(z, T):
        return 'All vapor'
    if P > bubbleP_NRTL(z,T):
        return 'All liquid'



In [58]:
T=373
P=(dewP_NRTL(z, T)[0]+bubbleP_NRTL(z,T)[0])/2

(215679.4631248687, array([0.02121585, 0.13299478, 0.17452966, 0.67125971]))

In [57]:
bubbleP_NRTL(z,T)

(Array(257793.05382014, dtype=float64),
 Array([0.26349637, 0.18347614, 0.27717804, 0.27584945], dtype=float64))